<a href="https://colab.research.google.com/github/xarvel/DataScience/blob/master/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 1s 1us/step


In [3]:
def text_stats(text):
  sample_size = 250
  print(f'Sample {sample_size} characters:')
  print('-' * 80)
  # Take a look at the first 250 characters in text
  print(text[:sample_size])
  print('-' * 80)
  # length of text is the number of characters in it
  print(f'Length of text: {len(text)} characters')
  # The unique characters in the file
  vocab = sorted(set(text))
  print(f'{len(vocab)} unique characters')

  return vocab

In [4]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
vocab = text_stats(text)

Sample 250 characters:
--------------------------------------------------------------------------------
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

--------------------------------------------------------------------------------
Length of text: 1115394 characters
65 unique characters


In [5]:
SEQUENCE_LENGTH = 100

# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

# The embedding dimension
EMBEDDING_DIMENTION = 256

# Number of RNN units
RNN_UNITS = 1024

In [6]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab),
    mask_token=None
)
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(),
    invert=True,
    mask_token=None
)

In [7]:
# Length of the vocabulary in StringLookup Layer
VOCAB_SIZE = len(ids_from_chars.get_vocabulary())

In [8]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [9]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
sequences = ids_dataset.batch(SEQUENCE_LENGTH + 1, drop_remainder=True)
dataset = sequences.map(split_input_target)

In [10]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [11]:
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [12]:
class RNNModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(
        rnn_units,
        return_sequences=True,
        return_state=True
    )
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [13]:
model = RNNModel(
    vocab_size=VOCAB_SIZE,
    embedding_dim=EMBEDDING_DIMENTION,
    rnn_units=RNN_UNITS
)

In [14]:
model.build((SEQUENCE_LENGTH, VOCAB_SIZE))
model.summary()

Model: "rnn_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [16]:
model.compile(optimizer='adam', loss=loss)

In [17]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [18]:
EPOCHS = 20

In [19]:
mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

Epoch 1 Batch 0 Loss 4.1893
Epoch 1 Batch 50 Loss 2.8173
Epoch 1 Batch 100 Loss 2.3543
Epoch 1 Batch 150 Loss 2.2344

Epoch 1 Loss: 2.7151
Time taken for 1 epoch 23.25 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 2.1962
Epoch 2 Batch 50 Loss 2.0573
Epoch 2 Batch 100 Loss 1.9414
Epoch 2 Batch 150 Loss 1.8585

Epoch 2 Loss: 1.9913
Time taken for 1 epoch 10.43 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.7640
Epoch 3 Batch 50 Loss 1.7264
Epoch 3 Batch 100 Loss 1.6562
Epoch 3 Batch 150 Loss 1.6445

Epoch 3 Loss: 1.7120
Time taken for 1 epoch 11.05 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.6201
Epoch 4 Batch 50 Loss 1.5507
Epoch 4 Batch 100 Loss 1.5321
Epoch 4 Batch 150 Loss 1.5451

Epoch 4 Loss: 1.5509
Time taken for 1 epoch 10.01 sec
_____________________________________________________________________

In [20]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [21]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [22]:
def predict_text(start_text, size):
  start = time.time()
  states = None
  next_char = tf.constant([start_text])
  result = [next_char]

  for n in range(size):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

  result = tf.strings.join(result)
  end = time.time()
  print(result[0].numpy().decode('utf-8'), '\n\n' + '_' * 80)
  print('\nRun time:', end - start)

In [23]:
predict_text('ROMEO:', 1000)

ROMEO:
Not or alize and her understand.

DUKE VINCENTIO:
I had pinined it as you.

ARCHIDAMUS:
What, is new born to't?

PETRUCHIO:
A sign of kings, to choose me for thy fault's proud,
But, like a traitor and a heavier traitor.

KING HENRY VI:
For which no wrong more-spirit, made your manlets;
List not a kinsman: noble frozens in
the infirmity of sleep, alack, this face,
And nettles precious little better curse
Hath Hadly more a single trick
For time of desperate ancient to our hair
Below this spounding throne an oak,
And heaven's sightly service, which I do remember
Myself that could we adjudge their lives
Brought him from man of my owlings; therefore, now,
Now forsworn! that fetch shall find Bohemia's woe.

KING EDWARD IV:
Bawe, friar! Give me a ledge? cry:
It was his grave in matter that must I think,
To crush an old marcius' man, I'll help me
To the sound corpers' treason: he is past
Am I a sight in Argination.

FLORIZEL:
Strength,
To sing and bark at once, you fain deny could'st,
A

In [24]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')